In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit, GroupKFold

In [ ]:
df1 = pd.read_csv('patras_data.csv', parse_dates=['date'])
df2 = pd.read_csv('sheffield_data.csv', parse_dates=['date'])

# Helper functions

In [ ]:
def imputer(df, lookup_table, median_val, variable):
    '''
    If zero height value, impute mean height value depending on patient's gender/ethnicity combination.
    If patient's gender/ethnicity combination has no non-zero mean height value, impute the mean height value of all patients with non-zero heights.
    '''
    df2 = df.copy()
    for i in range(len(df)):
        if df2.at[i, variable] == 0:
            for j in range(len(lookup_table)):
                if df2.at[i, 'gender'] == lookup_table.at[j, 'gender'] and df2.at[i, 'ethnicity'] == lookup_table.at[j, 'ethnicity']:
                    df2.at[i, variable] = lookup_table.at[j, variable]
                    break
            else:
                df2.at[i, variable] = median_val
    
    return df2

In [ ]:
def shuffle_data_by_group(df, group_column_name, random_state=1):
    random.seed(random_state)
    group_object = df.groupby(group_column_name)
    groups = [group_object.get_group(x) for x in group_object.groups]
    random.shuffle(groups)
    for i in range(len(groups)):
        groups[i][group_column_name] = i
    return pd.concat(groups).reset_index(drop=True)

In [ ]:
def reset_group_id(df, group_column_name):
    group_object = df.groupby(group_column_name)
    groups = [group_object.get_group(x) for x in group_object.groups]
    for i in range(len(groups)):
        groups[i][group_column_name] = i
    return pd.concat(groups).reset_index(drop=True)

In [ ]:
def datetime_to_days_diff(df, group_column_name, time_column_name):
    group_object = df.groupby(group_column_name)
    grouped_data = [group_object.get_group(x) for x in group_object.groups]
    new_group_list = []
    for group in grouped_data:
        group['times'] = None
        ref = group[time_column_name].iloc[0]
        for i in range(len(group)):
            group['times'].iloc[i] = (group[time_column_name].iloc[i] - ref) / np.timedelta64(1, 'D')
        group['times'] = pd.to_numeric(group['times'], downcast="integer")
        new_group_list.append(group)
    df_new = pd.concat(new_group_list, axis=0, ignore_index=True)
    return df_new

In [ ]:
def train_test_split_grouped_interpolation(df, group_sizes, test_size=0.2, random_state=1):
    '''
    Train/test split, but test set contains at least one observation from each group in the training set, and contains no unseen groups.
    '''
    assert 0 < test_size < 1, "Test size must be strictly between 0 and 1"
    assert np.sum(group_sizes) == len(df), "Sum of group_sizes must be equal to length dataframe"
    assert group_sizes.all() > 0, "Group sizes should be non-negative"
    assert len(group_sizes) < len(df), "Number of groups should be less than number of observations"

    np.random.seed(random_state)
    df_len = len(df)
    test_len = int(test_size * df_len)
    no_groups = len(group_sizes)

    # Pick one observation from all groups
    sample_len = no_groups
    n_samples_chosen_per_group = np.ones_like(group_sizes)
    last_idx_arr = np.cumsum(group_sizes)-1 # Array of index of the last observation in each group within the overall dataset
    test_idx = [last_idx_arr[i] for i in range(no_groups)]
    group_sizes_new = group_sizes.copy()
    group_sizes_new -= 1

    # Keep picking more observations until the required number of test observations has been picked
    while sample_len < test_len:
        group_idx = np.random.randint(no_groups) # Pick a random group
        if group_sizes_new[group_idx] > 1:
            if test_len - sample_len > 1:
                n = np.random.randint(1, min([group_sizes_new[group_idx], test_len-sample_len])) # Pick a random sample of size 1<=n<group_size from the chosen group
            else:
                n = 1
            last_idx = last_idx_arr[group_idx]-n_samples_chosen_per_group[group_idx] # Index of the last observation remaining in each group within the overall dataset
            test_idx += [last_idx-i for i in range(n)]
            n_samples_chosen_per_group[group_idx] += n # Update number of samples chosen from the group
            group_sizes_new[group_idx] -= n # Update current group sizes
            sample_len += n

    mask = np.ones(df_len, dtype=bool)
    mask[test_idx] = False
    df_train, df_test = df[mask], df[~mask]

    return df_train, df_test, n_samples_chosen_per_group

In [ ]:
def train_test_split_grouped_extrapolation(df, groups, test_size=0.2, random_state=1):
    '''
    Train/test split for a dataframe, but test set only contains only unseen groups.
    ``test_size`` represents the proportion of groups to include in the test split (rounded up).
    '''
    train_idx, test_idx = next(GroupShuffleSplit(test_size=test_size, random_state=random_state).split(df, groups=groups))
    df_train, df_test = df.iloc[train_idx], df.iloc[test_idx]

    return df_train, df_test

# Combine dataframes

In [ ]:
# Reset Patient ID for both dataframes
df1.groupby(['ID']).ngroup()

0         0
1         0
2         0
3         0
4         0
       ... 
4348    558
4349    558
4350    558
4351    559
4352    559
Length: 4353, dtype: int64

In [ ]:
df1['ID'] = df1.groupby(['ID']).ngroup()

In [ ]:
df2.groupby(['ID']).ngroup() + df1['ID'].unique().size

0        560
1        560
2        560
3        560
4        560
        ... 
6637    1552
6638    1552
6639    1553
6640    1553
6641    1553
Length: 6642, dtype: int64

In [ ]:
df2['ID'] = df2.groupby(['ID']).ngroup() + df1['ID'].unique().size

In [ ]:
data = pd.concat([df1, df2], axis=0, ignore_index=True)

In [ ]:
data

,ID,site,date_entered_study,control_patient,dob_year,gender,ethnicity,employment,height,smoker,kidney_transplant,kidney_transplant_Date,Patient_died,Date_of_death,disease,CGN_disease,sampleId,date,egfr,ckd_stage,slope_allprior,N_allprior,Class_allprior,slope_2yprior,N_2yprior,Class_2yprior,bp.di,bp.sys,sc,bun,X24h.pr,sampletaken,ace_inhibitors,arbs,statins,rate_decline_egfr,urine_creatinine,one_over_slope,serum_cholestrol,serum_triglycerides,serum_calcium,serum_phosphorus,serum_ca_p,serum_PTH,protein_urea,serum_creatinin,urine_albumin,urineprotein_creat_ratio,urinealbumin_crea_ratio,pulse,Weight,bmi,bsa,waist_circum,hip_circum,midarm_circum,triceps_skinfold,diabetic,mt_glom_scler,mt_tub_fib,mt_vas_scler,serum_glucose,haematuria,micro_albuminuria,hba1c,c_reactive_protein,retinopathy,neuropathy,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe,AKI,Started_dialysis
0,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2011-04-10,69.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,80.0,150.0,1.1,47.0,1250.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2012-07-02,62.0,NaN,-20.277778,2.0,Progressor,-20.277778,2.0,Progressor,0.0,0.0,1.2,64.0,2058.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2012-11-09,56.0,NaN,-13.424735,3.0,Progressor,-13.424735,3.0,Progressor,85.0,145.0,1.3,64.0,1993.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2013-03-26,62.0,NaN,-4.958831,4.0,Slow_progressor,-4.958831,4.0,Slow_progressor,90.0,160.0,1.2,55.0,1960.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2013-09-17,50.0,NaN,-7.323452,5.0,Progressor,-7.323452,5.0,Progressor,100.0,165.0,1.5,54.0,3936.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10990,1552,Sheffield,9/6/19,False,1973.0,Female,Caucasian,NaN,0.0,NaN,False,NaN,False,NaN,Other,Other,NaN,2019-06-13,65.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,95.0,137.0,91.0,5.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10991,1552,Sheffield,9/6/19,False,1973.0,Female,Caucasian,NaN,0.0,NaN,False,NaN,False,NaN,Other,Other,2164.0,2019-12-09,64.0,0.0,-4.010989,2.0,Slow_progressor,-4.010989,2.0,Slow_progressor,81.0,139.0,NaN,4.5,0.0,1,False,False,False,-4.010989,117.647059,-0.000479,0.0,0.0,2.28,0.97,2.2116,0.0,0.07,92.0,4.7,595.0,3995.0,NaN,76.6,NaN,0.0,NaN,NaN,NaN,NaN,False,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
10992,1553,Sheffield,9/6/19,False,1948.0,Female,Caucasian,NaN,0.0,NaN,False,NaN,

In [ ]:
data.describe()

,ID,dob_year,height,sampleId,egfr,ckd_stage,slope_allprior,N_allprior,slope_2yprior,N_2yprior,bp.di,bp.sys,sc,bun,X24h.pr,sampletaken,rate_decline_egfr,urine_creatinine,one_over_slope,serum_cholestrol,serum_triglycerides,serum_calcium,serum_phosphorus,serum_ca_p,serum_PTH,protein_urea,serum_creatinin,urine_albumin,urineprotein_creat_ratio,urinealbumin_crea_ratio,pulse,Weight,bmi,bsa,waist_circum,hip_circum,midarm_circum,triceps_skinfold,mt_glom_scler,mt_tub_fib,mt_vas_scler,serum_glucose,haematuria,micro_albuminuria,hba1c,c_reactive_protein,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe
count,10995.000000,8924.000000,10985.000000,3851.000000,10522.000000,3847.000000,9024.000000,10316.000000,8916.000000,10316.000000,10706.000000,10696.000000,7144.000000,10971.000000,10219.000000,10995.00000,3720.000000,3850.000000,3688.000000,3394.000000,3392.000000,3785.000000,3645.000000,3731.000000,3009.000000,3849.000000,3848.000000,3847.000000,2.956000e+03,2.864000e+03,1101.000000,3292.000000,1664.000000,3266.000000,1.0,0.0,0.0,1.0,3851.000000,3851.000000,3851.000000,579.000000,159.000000,187.000000,286.000000,234.000000,603.000000,599.000000,599.000000,155.000000,154.000000,340.000000,292.000000
mean,704.031924,1959.437584,5.387872,1016.242534,46.771944,1.408630,0.034643,5.451241,-0.054014,4.063494,57.916589,102.647532,111.372924,96.484386,359.728078,0.35025,-6.660000,63.454642,-3.712751,66.141956,52.363178,5.020383,1.970879,13.003042,29.090592,41.151716,82.552412,309.181619,8.996384e+03,1.160583e+05,73.314260,103.689894,32.355805,-3.887033,54.0,NaN,NaN,70.0,0.512942,0.033757,0.042067,84.280829,1.943396,1555.666695,23.944965,2.914231,57.589552,2628.758097,1514.981636,410.993548,9.491558,48.803235,207.782192
std,459.267671,15.180295,27.448790,601.277980,25.192801,1.722691,67.007580,3.621561,68.600263,2.270368,38.361888,64.578256,126.832653,6614.112817,1499.422438,0.47707,70.896595,70.497815,18.916902,94.353686,90.396834,5.295327,3.824270,36.915358,68.770020,144.344466,109.340401,937.586182,2.463619e+05,1.871267e+06,25.636006,1177.404809,508.161163,22.820254,NaN,NaN,NaN,NaN,4.431075,0.247382,0.306291,77.081517,7.628432,2309.599102,26.783405,9.077228,55.430334,3926.904331,18566.007661,235.477892,22.940807,37.259726,125.553761
min,0.000000,1926.000000,0.000000,1.000000,0.000000,0.000000,-1460.000000,1.000000,-1460.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-3504.000000,0.000000,-99.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-8.840000e+03,0.000000,0.000000,-99.990000,-99.990000,54.0,NaN,NaN,70.0,-1.000000,-1.000000,-1.000000,1.400000,0.000000,0.000000,4.600000,0.000000,7.900000,3.700000,22.000000,50.000000,1.700000,4.300000,3.800000
25%,276.000000,1948.000000,0.000000,510.500000,27.000000,0.000000,-5.318589,2.000000,-5.903179,2.000000,0.000000,0.000000,1.600000,8.000000,0.000000,0.00000,-5.526511,32.805430,-0.002939,0.000000,0.000000,2.290000,0.880000,1.715750,0.000000,0.070000,1.000000,0.000000,1.653280e+02,0.000000e+00,68.000000,69.200000,23.053422,0.000000,54.0,NaN,NaN,70.0,0.000000,0.000000,0.000000,10.300000,0.000000,58.750000,6.400000,0.230000,11.900000,7.290000,187.500000,265.500000,4.025000,14.950000,54.200000
50%,711.000000,1957.000000,1.560000,997.000000,42.000000,0.000000,-1.031236,5.000000,-1.170697,4.000000,73.000000,130.000000,108.500000,17.000000,0.000000,0.00000,-1.444804,55.450000,-0.000226,0.000000,0.000000,2.450000,1.250000,2.702500,0.000000,0.700000,2.600000,4.000000,9.668750e+02,7.091464e+01,70.000000,79.850000,25.887944,0.000000,54.0,NaN,NaN,70.0,0.000000,0.000000,0.000000,87.000000,0.000000,545.000000,7.750000,0.435000,14.600000,9.820000,238.000000,379.000000,5.550000,46.000000,254.500000
75%,1098.000000,1969.000000,1.700000,1481.500000,61.000000,3.000000,2.282602,8.000000,3.055405,5.000000,82.000000,146.000000,178.000000,50.000000,0.000000,1.00000,1.458917,84.025000,0.001257,157.000000,97.000000,9.200000,3.200000,27.55000

In [ ]:
# Select non-control patients
data = data[data['control_patient']==False]
data.reset_index(inplace=True, drop=True)
data

,ID,site,date_entered_study,control_patient,dob_year,gender,ethnicity,employment,height,smoker,kidney_transplant,kidney_transplant_Date,Patient_died,Date_of_death,disease,CGN_disease,sampleId,date,egfr,ckd_stage,slope_allprior,N_allprior,Class_allprior,slope_2yprior,N_2yprior,Class_2yprior,bp.di,bp.sys,sc,bun,X24h.pr,sampletaken,ace_inhibitors,arbs,statins,rate_decline_egfr,urine_creatinine,one_over_slope,serum_cholestrol,serum_triglycerides,serum_calcium,serum_phosphorus,serum_ca_p,serum_PTH,protein_urea,serum_creatinin,urine_albumin,urineprotein_creat_ratio,urinealbumin_crea_ratio,pulse,Weight,bmi,bsa,waist_circum,hip_circum,midarm_circum,triceps_skinfold,diabetic,mt_glom_scler,mt_tub_fib,mt_vas_scler,serum_glucose,haematuria,micro_albuminuria,hba1c,c_reactive_protein,retinopathy,neuropathy,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe,AKI,Started_dialysis
0,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2011-04-10,69.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,80.0,150.0,1.1,47.0,1250.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2012-07-02,62.0,NaN,-20.277778,2.0,Progressor,-20.277778,2.0,Progressor,0.0,0.0,1.2,64.0,2058.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2012-11-09,56.0,NaN,-13.424735,3.0,Progressor,-13.424735,3.0,Progressor,85.0,145.0,1.3,64.0,1993.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2013-03-26,62.0,NaN,-4.958831,4.0,Slow_progressor,-4.958831,4.0,Slow_progressor,90.0,160.0,1.2,55.0,1960.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Patras,16/2/16 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,False,NaN,False,NaN,DN,DN,NaN,2013-09-17,50.0,NaN,-7.323452,5.0,Progressor,-7.323452,5.0,Progressor,100.0,165.0,1.5,54.0,3936.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10814,1552,Sheffield,9/6/19,False,1973.0,Female,Caucasian,NaN,0.0,NaN,False,NaN,False,NaN,Other,Other,NaN,2019-06-13,65.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,95.0,137.0,91.0,5.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10815,1552,Sheffield,9/6/19,False,1973.0,Female,Caucasian,NaN,0.0,NaN,False,NaN,False,NaN,Other,Other,2164.0,2019-12-09,64.0,0.0,-4.010989,2.0,Slow_progressor,-4.010989,2.0,Slow_progressor,81.0,139.0,NaN,4.5,0.0,1,False,False,False,-4.010989,117.647059,-0.000479,0.0,0.0,2.28,0.97,2.2116,0.0,0.07,92.0,4.7,595.0,3995.0,NaN,76.6,NaN,0.0,NaN,NaN,NaN,NaN,False,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
10816,1553,Sheffield,9/6/19,False,1948.0,Female,Caucasian,NaN,0.0,NaN,False,NaN,

# Cleaning not involving imputing values

## Initial cleaning

In [ ]:
# Dataframe after selecting relevant columns
data2 = data[['ID', 'site', 'dob_year','gender', 'ethnicity', 'height', 'Weight', 'smoker', 'kidney_transplant', 'Patient_died', 'disease', 'bp.sys', 'bun', 'date', 'egfr']]
data2 = data2.rename(columns={'Weight': 'weight', 'Patient_died': 'patient_died'})
data2['ID'] = data2.groupby(['ID']).ngroup()
data2

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10814,1421,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10815,1421,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10816,1422,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10817,1422,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


In [ ]:
data2['smoker'] = data2['smoker'].replace({'Past Smoker': 'Past-Smoker'})

In [ ]:
data2['smoker'].value_counts()

Non-Smoker     2602
Past-Smoker    1567
Smoker          896
Name: smoker, dtype: int64

In [ ]:
# Notice that for this patient, egfr and ckd_stage appears to have been switched incorrectly, so we manualy set egfr to the correct value
data.loc[data['egfr']==3]

,ID,site,date_entered_study,control_patient,dob_year,gender,ethnicity,employment,height,smoker,kidney_transplant,kidney_transplant_Date,Patient_died,Date_of_death,disease,CGN_disease,sampleId,date,egfr,ckd_stage,slope_allprior,N_allprior,Class_allprior,slope_2yprior,N_2yprior,Class_2yprior,bp.di,bp.sys,sc,bun,X24h.pr,sampletaken,ace_inhibitors,arbs,statins,rate_decline_egfr,urine_creatinine,one_over_slope,serum_cholestrol,serum_triglycerides,serum_calcium,serum_phosphorus,serum_ca_p,serum_PTH,protein_urea,serum_creatinin,urine_albumin,urineprotein_creat_ratio,urinealbumin_crea_ratio,pulse,Weight,bmi,bsa,waist_circum,hip_circum,midarm_circum,triceps_skinfold,diabetic,mt_glom_scler,mt_tub_fib,mt_vas_scler,serum_glucose,haematuria,micro_albuminuria,hba1c,c_reactive_protein,retinopathy,neuropathy,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe,AKI,Started_dialysis
3539,373,Patras,17/7/17 12:39,False,1956.0,Male,Caucasian,Semi-Professional,1.7,Smoker,False,NaN,False,NaN,DN,DN,1042.0,2017-04-12,3.0,41.0,-74.118006,3.0,Progressor,-74.118006,3.0,Progressor,100.0,170.0,NaN,73.0,1639.0,1,False,True,True,-74.118006,66.26,-0.162154,NaN,NaN,10.0,4.5,0.0,NaN,71.3,1.8,475.0,NaN,NaN,70.0,80.0,NaN,-99.99,NaN,NaN,NaN,NaN,True,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [ ]:
data2.loc[data2['egfr']==3, 'egfr'] = 41

In [ ]:
# Sort values by date
data3 = data2.sort_values(['ID', 'date'], ignore_index=True)

## Regrouping ethnicity

In [ ]:
data4 = data3.copy()

In [ ]:
data4['ethnicity'].value_counts()

Caucasian                        9209
Asian                             146
Black (afro caribean descent)     108
Others                             56
Asian - Indian                     12
INDIAN                             11
Asian (Pakistan)                   11
Somali                              9
Chinese                             6
Asian(INDIAN)                       6
Indian                              5
Asian( PAKISTANI)                   5
Asian ( BANGLADESHHI)               2
Arab                                1
Name: ethnicity, dtype: int64

In [ ]:
# Combine categories
data4['ethnicity'] = data4['ethnicity'].replace(dict.fromkeys(['Asian - Indian', 'Asian (Pakistan)', 'INDIAN', 'Chinese', 'Asian(INDIAN)', 'Indian', 'Asian( PAKISTANI)', 'Arab', 'Asian ( BANGLADESHHI)'], 'Asian'))
data4['ethnicity'] = data4['ethnicity'].replace({'Somali': 'Others'})
data4['ethnicity'] = data4['ethnicity'].replace({'Black (afro caribean descent)': 'Black'})

In [ ]:
data4['ethnicity'].value_counts()

Caucasian    9209
Asian         205
Black         108
Others         65
Name: ethnicity, dtype: int64

## egfr

In [ ]:
data4['egfr'].unique()

array([ 69. ,  62. ,  56. ,  50. ,  54. ,  53. ,  46. ,  58. ,  49. ,
        90. , 103. ,  33. ,  31. ,  29. ,  36. ,  35. ,  45. ,  48. ,
        59. ,  72. ,  41. ,  44. ,  64. ,  60. ,  71. ,  47. ,  66. ,
        76. ,  52. ,  39. ,  28. ,  84. ,  89. ,  24. ,  38. ,  27. ,
        21. ,  22. ,  30. ,  23. ,  20. ,  19. ,  26. ,  16. ,  13. ,
        32. ,  34. ,  43. ,  37. ,  25. ,   9. ,  15. ,  10. , 107. ,
        82. ,  93. ,  63. , 106. ,  92. , 118. , 150. , 149. , 101. ,
       125. , 127. , 124. ,  67. ,  55. ,  40. ,  57. ,  68. ,  42. ,
        65. ,  73. ,  80. ,  81. ,  61. ,  18. ,  78. ,  70. ,  14. ,
        17. ,  11. ,  12. ,  85. ,  77. ,  79. , 102. ,  74. ,  51. ,
        97. ,  83. ,  96. , 108. ,  94. ,  86. ,  75. , 109. ,  91. ,
       104. ,  88. ,  87. ,  95. , 114. , 100. , 133. ,  99. , 113. ,
       112. ,   7. , 120. ,  98. , 158. , 128. , 129. , 145. , 122. ,
       111. , 117. , 110. , 131. ,   8. , 123. , 146. , 105. , 136. ,
       119. , 141. ,

In [ ]:
# Missing values for egfr
data4['egfr'].isna().sum()

402

In [ ]:
# Drop all rows with missing egfr values
data5 = data4.dropna(subset=['egfr'])
data5.reset_index(inplace=True, drop=True)
data5['ID'] = data5.groupby(['ID']).ngroup()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# Drop row with 0 egfr value (1 row)
data6 = data5.loc[data5['egfr'] != 0]
data6.reset_index(inplace=True, drop=True)
data6['ID'] = data6.groupby(['ID']).ngroup()
data6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10411,1370,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10412,1370,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10413,1371,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10414,1371,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


## Height

In [ ]:
# Some issues with height
data6.describe()

,ID,dob_year,height,weight,bp.sys,bun,egfr
count,10416.000000,8674.000000,10406.000000,3161.000000,10118.000000,10399.000000,10416.000000
mean,604.731663,1959.417454,5.495013,104.621711,106.653390,101.503183,46.569931
std,398.776884,15.168561,27.748251,1201.547418,62.594163,6793.564265,25.039977
min,0.000000,1926.000000,0.000000,1.480000,0.000000,0.000000,4.000000
25%,228.000000,1948.000000,0.000000,69.300000,100.000000,9.000000,27.000000
50%,616.000000,1957.000000,1.560000,79.700000,132.000000,18.000000,42.000000
75%,936.000000,1969.000000,1.700000,90.400000,147.000000,52.000000,61.000000
max,1371.000000,2002.000000,188.000000,67125.000000,1158.000000,692801.000000,189.000000


In [ ]:
data6['height'].unique()

array([  1.7  ,   1.67 ,   1.6  ,   1.68 ,   1.65 ,   1.63 ,   1.66 ,
         1.45 ,   1.64 ,   1.8  ,   1.75 ,   1.73 ,   1.62 ,   1.56 ,
         1.76 ,   1.74 ,   1.57 ,   1.71 ,   1.5  ,   1.78 ,   1.48 ,
         1.72 ,   1.69 ,   1.89 ,   1.52 ,   1.59 ,   1.58 ,   1.87 ,
         1.82 ,   1.77 ,   1.53 ,   1.44 ,   1.61 ,   0.   ,   1.95 ,
         1.54 ,   1.88 ,   1.81 ,   1.51 , 169.   ,   1.55 ,   1.9  ,
         2.   ,   1.42 ,   1.41 ,   1.47 ,   1.85 ,   1.83 ,   1.84 ,
           nan,   1.86 ,   1.79 ,   1.92 ,   1.91 , 158.   ,   1.601,
         1.752,   1.625,   1.98 , 180.   , 173.   ,   1.49 , 174.   ,
         2.02 ,  97.6  , 172.   , 166.   , 167.   , 171.   , 142.   ,
       181.   , 188.   , 176.   , 184.   , 168.   , 178.   , 179.   ,
       155.   , 182.   , 183.   , 165.   , 159.   ])

In [ ]:
data6['height'].isna().sum()

10

In [ ]:
data6[data6['height'].isna().to_numpy()]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
3858,364,Patras,1954.0,Female,Caucasian,NaN,76.0,Non-Smoker,False,False,GMN,125.0,53.0,2017-12-10,53.0
3859,364,Patras,1954.0,Female,Caucasian,NaN,90.0,Non-Smoker,False,False,GMN,130.0,50.0,2018-05-03,53.0
5636,667,Sheffield,1988.0,Male,Caucasian,NaN,NaN,NaN,False,False,HTN,142.0,9.0,2015-01-30,48.0
5637,667,Sheffield,1988.0,Male,Caucasian,NaN,NaN,NaN,False,False,HTN,NaN,10.0,2015-10-29,51.0
5638,667,Sheffield,1988.0,Male,Caucasian,NaN,NaN,NaN,False,False,HTN,145.0,10.0,2016-05-02,48.0
5639,667,Sheffield,1988.0,Male,Caucasian,NaN,73.6,NaN,False,False,HTN,154.0,11.3,2017-02-17,50.0
10216,1316,Sheffield,1946.0,Female,NaN,NaN,NaN,NaN,False,False,GMN,144.0,9.0,2018-05-24,24.0
10217,1316,Sheffield,1946.0,Female,NaN,NaN,NaN,NaN,False,False,GMN,149.0,11.0,2018-10-18,30.0
10218,1316,Sheffield,1946.0,Female,NaN,NaN,71.0,NaN,False,False,GMN,150.0,10.3,2019-03-28,34.0
10219,1316,Sheffield,1946.0,Female,NaN,NaN,69.2,NaN,False,False,GMN,144.0,14.0,2019-10-10,32.0


In [ ]:
# Remove NaN values
data7 = data6.dropna(subset=['height'])
data7.reset_index(inplace=True, drop=True)
data7['ID'] = data7.groupby(['ID']).ngroup()
data7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10401,1367,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10402,1367,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10403,1368,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10404,1368,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


In [ ]:
# Strange value
data7.loc[data7['height']==97.6]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
5994,711,Sheffield,NaN,Male,Asian,97.6,NaN,NaN,False,False,GMN,0.0,0.0,2010-06-14,108.0
5995,711,Sheffield,NaN,Male,Asian,97.6,NaN,NaN,False,False,GMN,0.0,0.0,2010-10-18,117.0
5996,711,Sheffield,NaN,Male,Asian,97.6,107.8,NaN,False,False,GMN,0.0,21.9,2010-11-10,18.0


In [ ]:
# Remove strange values
data8 = data7.drop(data7.loc[data7['height']==97.6].index.to_numpy())
data8.reset_index(inplace=True, drop=True)
data8['ID'] = data8.groupby(['ID']).ngroup()
data8

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10398,1366,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10399,1366,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10400,1367,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10401,1367,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


In [ ]:
# Convert all height measurements to meters
data8['height'] = np.where(data8['height'] > 5, data8['height'] / 100, data8['height'])

In [ ]:
data8['height'].unique()

array([1.7  , 1.67 , 1.6  , 1.68 , 1.65 , 1.63 , 1.66 , 1.45 , 1.64 ,
       1.8  , 1.75 , 1.73 , 1.62 , 1.56 , 1.76 , 1.74 , 1.57 , 1.71 ,
       1.5  , 1.78 , 1.48 , 1.72 , 1.69 , 1.89 , 1.52 , 1.59 , 1.58 ,
       1.87 , 1.82 , 1.77 , 1.53 , 1.44 , 1.61 , 0.   , 1.95 , 1.54 ,
       1.88 , 1.81 , 1.51 , 1.55 , 1.9  , 2.   , 1.42 , 1.41 , 1.47 ,
       1.85 , 1.83 , 1.84 , 1.86 , 1.79 , 1.92 , 1.91 , 1.601, 1.752,
       1.625, 1.98 , 1.49 , 2.02 ])

In [ ]:
# Too many rows with zero values for height to drop from the dataframe
print('Number of rows with zero height values: ',(data8['height']==0).sum())

# All patients with zero height values do not have previously-recorded height values
print('Do all patients with zero height values not have any previously-recorded height values? ', data8.groupby(['ID']).height.min().equals(data8.groupby(['ID']).height.max()))

# All patients with non-zero height values have the same height values for all of their individual patient-level observations (i.e. their heights do not change over time)
print('Do all patients with non-zero height values have the same height values for all of their individual patient-level observations (i.e. their heights do not change over time)? ', data8.loc[data8['height']!=0].groupby(['ID']).height.median().median() == data8.loc[data8['height']!=0].groupby(['ID']).first().height.median())

Number of rows with zero height values:  4761
Do all patients with zero height values not have any previously-recorded height values?  True
Do all patients with non-zero height values have the same height values for all of their individual patient-level observations (i.e. their heights do not change over time)?  True


## Age

In [ ]:
# All patients with zero dob_year values do not have previously-recorded values
data8.groupby(['ID']).dob_year.min().equals(data8.groupby(['ID']).dob_year.max())

True

In [ ]:
data9 = data8.copy()
data9['dob_year'].fillna(0, inplace=True)

## bp.sys

In [ ]:
data10 = data9.copy()

In [ ]:
# NaN, zero values and unusually small and large values
data10['bp.sys'].unique()

array([ 150.,    0.,  145.,  160.,  165.,  140.,  135.,  125.,  115.,
        120.,  180.,  130.,  110.,   nan,  155.,  137.,  122.,  105.,
        170.,  134.,  124.,  133.,  100.,  138.,  148.,  139.,  152.,
        121.,   96.,   95.,   90.,  200.,  136.,  128.,  127.,  132.,
        220.,  178.,  144.,  142.,   97.,  166.,  146.,  151.,  131.,
        117.,  126.,  300.,  123.,  175.,  149.,  190.,  156.,  114.,
         98.,  113.,  184.,  147.,  141.,  107.,   80.,  101.,  185.,
        195.,  154., 1158.,   14.,   85.,  106.,  143.,  164.,  103.,
        112.,  129.,  118.,  111.,  119.,  162.,  108.,  169.,  172.,
         92.,  177.,  153.,  159.,  174.,  116.,  173.,  168.,  183.,
        181.,  167.,  187.,  158.,  176.,  191.,  102.,  171.,  161.,
        163.,  188.,  192.,  157.,  196.,   91.,  199.,   88.,  109.,
        179.,   72.,  104.,  182.,  186.,  189.,  193.,  206.,  197.,
        209.,   17.,  237.,  202.,   99.,   93.,  215.,  225.,  207.,
        203.,  214.,

In [ ]:
data10.loc[data10['bp.sys']==1158]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2044,168,Patras,1956.0,Male,Caucasian,1.63,67.0,Past-Smoker,False,False,GMN,1158.0,34.0,2016-12-07,122.0


In [ ]:
data10.iloc[2043:2046, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2043,168,Patras,1956.0,Male,Caucasian,1.63,NaN,Past-Smoker,False,False,GMN,120.0,34.0,2016-02-16,122.0
2044,168,Patras,1956.0,Male,Caucasian,1.63,67.0,Past-Smoker,False,False,GMN,1158.0,34.0,2016-12-07,122.0
2045,168,Patras,1956.0,Male,Caucasian,1.63,71.0,Past-Smoker,False,False,GMN,120.0,32.0,2017-01-31,146.0


In [ ]:
# Change strange value
data10.loc[data10['bp.sys']==1158, ['bp.sys']] = 120.0

In [ ]:
data10.loc[data10['bp.sys']==300]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
859,69,Patras,1952.0,Male,Caucasian,1.74,NaN,Past-Smoker,True,False,Transplant,300.0,25.0,2015-03-26,72.0


In [ ]:
data10.iloc[858:861, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
858,69,Patras,1952.0,Male,Caucasian,1.74,NaN,Past-Smoker,True,False,Transplant,120.0,39.0,2014-09-22,65.0
859,69,Patras,1952.0,Male,Caucasian,1.74,NaN,Past-Smoker,True,False,Transplant,300.0,25.0,2015-03-26,72.0
860,69,Patras,1952.0,Male,Caucasian,1.74,NaN,Past-Smoker,True,False,Transplant,128.0,42.0,2015-05-10,72.0


In [ ]:
# Change strange value
data10.loc[data10['bp.sys']==300, ['bp.sys']] = 120.0

In [ ]:
data10.loc[data10['bp.sys']==17]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
5186,612,Sheffield,0.0,Male,Caucasian,1.78,NaN,Non-Smoker,False,False,DN,17.0,17.0,2008-03-10,23.0


In [ ]:
data10.iloc[5185:5188, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
5185,612,Sheffield,0.0,Male,Caucasian,1.78,NaN,Non-Smoker,False,False,DN,160.0,17.0,2007-05-18,19.0
5186,612,Sheffield,0.0,Male,Caucasian,1.78,NaN,Non-Smoker,False,False,DN,17.0,17.0,2008-03-10,23.0
5187,612,Sheffield,0.0,Male,Caucasian,1.78,NaN,Non-Smoker,False,False,DN,127.0,15.0,2008-04-01,24.0


In [ ]:
# Change strange value
data10.loc[data10['bp.sys']==17, ['bp.sys']] = 160.0

In [ ]:
data10.loc[data10['bp.sys']==14]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2057,169,Patras,1954.0,Female,Caucasian,1.58,NaN,Non-Smoker,False,False,GMN,14.0,31.0,2015-09-06,60.0


In [ ]:
data10.iloc[2056:2059, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2056,169,Patras,1954.0,Female,Caucasian,1.58,NaN,Non-Smoker,False,False,GMN,140.0,42.0,2015-01-27,77.0
2057,169,Patras,1954.0,Female,Caucasian,1.58,NaN,Non-Smoker,False,False,GMN,14.0,31.0,2015-09-06,60.0
2058,169,Patras,1954.0,Female,Caucasian,1.58,NaN,Non-Smoker,False,False,GMN,110.0,37.0,2015-09-29,60.0


In [ ]:
# Change strange value
data10.loc[data10['bp.sys']==14, ['bp.sys']] = 140.0

In [ ]:
# Replace NaN with zero for now
data10['bp.sys'].replace(0, np.nan, inplace=True)

## bun

In [ ]:
data11 = data10.copy()

In [ ]:
# NaN, zero values and unusually large values
# a = data11['bun'].unique()
# np.sort(a)

In [ ]:
data11.loc[data11['bun']==692801]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
1851,151,Patras,1972.0,Male,Caucasian,1.71,NaN,Non-Smoker,False,False,DN,90.0,692801.0,2015-07-29,23.0


In [ ]:
data11.iloc[1850:1853, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
1850,151,Patras,1972.0,Male,Caucasian,1.71,NaN,Non-Smoker,False,False,DN,NaN,80.0,2015-01-28,29.0
1851,151,Patras,1972.0,Male,Caucasian,1.71,NaN,Non-Smoker,False,False,DN,90.0,692801.0,2015-07-29,23.0
1852,151,Patras,1972.0,Male,Caucasian,1.71,NaN,Non-Smoker,False,False,DN,101.0,72.0,2015-09-09,25.0


In [ ]:
# Change strange value
data11.loc[data11['bun']==692801, ['bun']] = 80.0

In [ ]:
data11.loc[data11['bun']==365]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2048,168,Patras,1956.0,Male,Caucasian,1.63,69.0,Past-Smoker,False,False,GMN,125.0,365.0,2018-01-30,122.0


In [ ]:
data11.iloc[2047:2050, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2047,168,Patras,1956.0,Male,Caucasian,1.63,72.0,Past-Smoker,False,False,GMN,130.0,36.0,2017-09-26,122.0
2048,168,Patras,1956.0,Male,Caucasian,1.63,69.0,Past-Smoker,False,False,GMN,125.0,365.0,2018-01-30,122.0
2049,168,Patras,1956.0,Male,Caucasian,1.63,70.0,Past-Smoker,False,False,GMN,NaN,42.0,2018-02-10,104.0


In [ ]:
# Change strange value
data11.loc[data11['bun']==365, ['bun']] = 36.0

In [ ]:
data11.loc[data11['bun']==471]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2107,174,Patras,1985.0,Female,Caucasian,1.57,NaN,Non-Smoker,False,False,GMN,NaN,471.0,2015-01-06,105.0


In [ ]:
data11.iloc[2106:2109, :]

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
2106,173,Patras,1966.0,Male,Caucasian,1.67,103.0,Past-Smoker,False,False,HTN,135.0,166.0,2019-07-31,11.0
2107,174,Patras,1985.0,Female,Caucasian,1.57,NaN,Non-Smoker,False,False,GMN,NaN,471.0,2015-01-06,105.0
2108,174,Patras,1985.0,Female,Caucasian,1.57,NaN,Non-Smoker,False,False,GMN,NaN,60.0,2015-03-30,105.0


In [ ]:
# Change strange value
data11.loc[data11['bun']==471, ['bun']] = 60.0

In [ ]:
# Replace NaN with zero for now
data11['bun'].replace(0, np.nan, inplace=True)

## Weight

In [ ]:
# No zero values for weight, but many NaN values
print(data11['weight'].isna().sum())
data11.loc[data11['weight']==0]

7248


,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr


In [ ]:
data12 = pd.concat([data11.loc[(data11['weight'] > 30) & (data11['weight'] < 200)], data11.loc[data11['weight'].isna()]]).reset_index(drop=True)
data12['ID'] = data12.groupby(['ID']).ngroup()
data12.sort_values(['ID', 'date'], ignore_index=True, inplace=True)

# Cleaning involving imputing values
We process data in the following data sets **separately** to avoid data leak:


*   Test (extrapolation): "test_extrapolation"
*   Test (interpolation): "test_interpolation"
*   Full training set: "data_train_full"

For 5-fold cross-validation:
*   Train set 1, Val set 1
*   Train set 2, Val set 2
*   Train set 3, Val set 3
*   Train set 4, Val set 4
*   Train set 5, Val set 5

stored in "train_set_list", "val_set_list"

For validation (Deep ME):
*   Train set: "data_train"
*   Val set: "data_val"

All data in test and validation sets are imputed based solely on the training set(s)

In [ ]:
# Convert dates to number of days since first observation for each patient
data13 = datetime_to_days_diff(data12, 'ID', 'date')

# Shuffle data by patient
data14 = shuffle_data_by_group(data13, 'ID', random_state=1)

## Create main train, test extrapolation, test interpolation sets

In [ ]:
# Get patients with only 1 observation, 80 in total
a = data14.groupby(['ID']).size()==1
single_obs_rows = data14[data14['ID'].map(a)]
single_obs_rows

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
94,12,Patras,1944.0,Male,Caucasian,1.69,67.0,Smoker,False,False,Other,115.0,56.0,2017-11-14,45.0,0
296,43,Patras,1981.0,Male,Caucasian,1.74,78.0,Past-Smoker,False,False,GMN,140.0,54.0,2017-08-11,61.0,0
514,68,Patras,1972.0,Female,Caucasian,1.58,60.0,NaN,False,False,GMN,125.0,60.0,2018-06-11,43.0,0
709,97,Patras,1944.0,Male,Caucasian,1.73,78.0,NaN,False,False,Other,NaN,99.0,2016-09-27,16.0,0
897,122,Patras,1954.0,Female,Caucasian,1.56,84.0,Non-Smoker,False,False,HTN,125.0,100.0,2017-11-22,48.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9648,1271,Patras,1930.0,Female,Caucasian,1.65,75.0,Non-Smoker,False,False,GMN,NaN,104.0,2019-11-19,20.0,0
9787,1288,Patras,1947.0,Male,Caucasian,1.75,78.0,NaN,False,False,Other,NaN,46.0,2016-10-20,44.0,0
10048,1324,Patras,1997.0,Female,Caucasian,1.61,62.0,Non-Smoker,False,False,GMN,120.0,33.0,2019-04-16,112.0,0
10049,1325,Patras,1947.0,Male,Caucasian,1.78,75.0,Past-Smoker,False,False,GMN,110.0,78.0,2020-04-28,37.0,0


In [ ]:
# Remaining data
data15 = data14[data14['ID'].map(~a)]

# Number of remaining groups
data15['ID'].unique().size

1287

In [ ]:
# Split test extrapolation set
data_temp, test_extrapolation = train_test_split_grouped_extrapolation(data15, data15['ID'], test_size=0.2, random_state=1)

In [ ]:
test_extrapolation = pd.concat([test_extrapolation, single_obs_rows]).reset_index(drop=True)
test_extrapolation

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
0,6,Patras,1945.0,Male,Caucasian,1.67,NaN,Non-Smoker,False,False,DN,NaN,88.0,2012-03-05,36.0,0
1,6,Patras,1945.0,Male,Caucasian,1.67,NaN,Non-Smoker,False,False,DN,NaN,107.0,2013-01-29,40.0,330
2,6,Patras,1945.0,Male,Caucasian,1.67,NaN,Non-Smoker,False,False,DN,NaN,94.0,2013-05-27,43.0,448
3,6,Patras,1945.0,Male,Caucasian,1.67,NaN,Non-Smoker,False,False,DN,NaN,79.0,2013-09-20,38.0,564
4,6,Patras,1945.0,Male,Caucasian,1.67,NaN,Non-Smoker,False,False,DN,NaN,45.0,2014-01-10,46.0,676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,1271,Patras,1930.0,Female,Caucasian,1.65,75.0,Non-Smoker,False,False,GMN,NaN,104.0,2019-11-19,20.0,0
2231,1288,Patras,1947.0,Male,Caucasian,1.75,78.0,NaN,False,False,Other,NaN,46.0,2016-10-20,44.0,0
2232,1324,Patras,1997.0,Female,Caucasian,1.61,62.0,Non-Smoker,False,False,GMN,120.0,33.0,2019-04-16,112.0,0
2233,1325,Patras,1947.0,Male,Caucasian,1.78,75.0,Past-Smoker,False,False,GMN,110.0,78.0,2020-04-28,37.0,0


In [ ]:
data_temp.reset_index(inplace=True, drop=True)
data_temp.sort_values(['ID', 'times'], ignore_index=True, inplace=True)
group_sizes = data_temp.groupby(['ID']).size().to_numpy()
data_train_full, test_interpolation, n_samples_chosen_per_group = train_test_split_grouped_interpolation(data_temp, group_sizes, test_size=0.25, random_state=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
test_interpolation.reset_index(inplace=True, drop=True)
test_interpolation

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
0,0,Sheffield,1955.0,Male,NaN,0.00,NaN,NaN,False,False,GMN,172.0,7.0,2017-02-23,75.0,1091
1,1,Sheffield,1959.0,Female,Caucasian,0.00,70.7,NaN,True,True,Transplant,174.0,10.0,2017-06-14,29.0,1223
2,2,Sheffield,1958.0,Male,Caucasian,0.00,65.2,NaN,True,False,Transplant,139.0,14.9,2019-03-20,26.0,1723
3,3,Sheffield,1972.0,Female,Caucasian,0.00,65.2,NaN,True,False,Transplant,154.0,6.4,2019-06-02,54.0,1458
4,4,Patras,1955.0,Male,Caucasian,1.75,80.0,Smoker,False,False,HTN,150.0,43.0,2016-05-10,72.0,374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,1363,Sheffield,0.0,Female,Caucasian,0.00,NaN,NaN,False,False,HTN,119.0,NaN,2010-09-07,19.0,816
2036,1363,Sheffield,0.0,Female,Caucasian,0.00,NaN,NaN,False,False,HTN,142.0,NaN,2010-10-15,16.0,854
2037,1364,Patras,1977.0,Male,Caucasian,1.65,86.0,Smoker,False,False,GMN,120.0,52.0,2017-12-06,60.0,1401
2038,1365,Sheffield,1949.0,Female,Caucasian,0.00,49.2,NaN,False,False,HTN,176.0,7.7,2019-11-07,35.0,1478


In [ ]:
data_train_full.reset_index(inplace=True, drop=True)
data_train_full

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
0,0,Sheffield,1955.0,Male,NaN,0.00,NaN,NaN,False,False,GMN,NaN,8.0,2014-02-28,73.0,0
1,0,Sheffield,1955.0,Male,NaN,0.00,NaN,NaN,False,False,GMN,143.0,8.0,2014-08-15,62.0,168
2,0,Sheffield,1955.0,Male,NaN,0.00,NaN,NaN,False,False,GMN,150.0,6.0,2014-12-19,74.0,294
3,0,Sheffield,1955.0,Male,NaN,0.00,NaN,NaN,False,False,GMN,138.0,7.0,2015-07-05,77.0,492
4,0,Sheffield,1955.0,Male,NaN,0.00,NaN,NaN,False,False,GMN,144.0,7.0,2015-10-29,70.0,608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,1366,Patras,1980.0,Male,Caucasian,1.78,68.0,Non-Smoker,True,False,Transplant,NaN,46.0,2017-03-14,66.0,275
6119,1366,Patras,1980.0,Male,Caucasian,1.78,68.0,Non-Smoker,True,False,Transplant,120.0,63.0,2017-06-06,61.0,359
6120,1366,Patras,1980.0,Male,Caucasian,1.78,68.0,Non-Smoker,True,False,Transplant,NaN,39.0,2017-09-28,66.0,473
6121,1366,Patras,1980.0,Male,Caucasian,1.78,68.0,Non-Smoker,True,False,Transplant,120.0,45.0,2017-11-29,61.0,535


## Create validation sets

In [ ]:
# Cross-validation sets
train_set_list, val_set_list = [], []
group_kfold = GroupKFold(n_splits=5)
for train_index, val_index in group_kfold.split(data_train_full, groups=data_train_full['ID']):
    train_set_list.append(data_train_full.iloc[train_index].reset_index(drop=True))
    val_set_list.append(data_train_full.iloc[val_index].reset_index(drop=True))

In [ ]:
# Training and validation sets
data_train, data_val = train_test_split_grouped_extrapolation(data_train_full, data_train_full['ID'], test_size=0.25, random_state=1)
data_train.reset_index(inplace=True, drop=True)
data_val.reset_index(inplace=True, drop=True)

In [ ]:
data_val

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
0,2,Sheffield,1958.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,108.0,12.0,2014-07-01,9.0,0
1,2,Sheffield,1958.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,96.0,23.0,2014-12-06,4.0,158
2,2,Sheffield,1958.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,153.0,10.0,2014-12-30,75.0,182
3,2,Sheffield,1958.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,NaN,10.0,2015-07-07,69.0,371
4,2,Sheffield,1958.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,117.0,8.0,2015-12-15,71.0,532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,1359,Sheffield,1958.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,130.0,7.0,2016-12-13,68.0,896
1708,1360,Sheffield,1966.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,140.0,16.0,2014-02-07,7.0,0
1709,1360,Sheffield,1966.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,134.0,4.0,2014-05-02,8.0,84
1710,1360,Sheffield,1966.0,Male,Caucasian,0.0,NaN,NaN,True,False,Transplant,134.0,6.0,2015-01-28,53.0,355


## Height

### Impute for data_train_full, test_extrapolation, test_interpolation

In [ ]:
# Median height among all non-zero rows
median_height = data_train_full.loc[data_train_full['height']!=0].groupby(['ID']).height.median().median()

# Median heights by gender and ethnicity among patients with non-zero height values
height_table = pd.DataFrame({'height': data_train_full.loc[data_train_full['height']!=0].groupby(['ID']).first().groupby(['gender', 'ethnicity']).height.median()}).reset_index()
height_table

In [ ]:
data_train_full2 = imputer(data_train_full, height_table, median_height, 'height')
test_extrapolation2 = imputer(test_extrapolation, height_table, median_height, 'height')
test_interpolation2 = imputer(test_interpolation, height_table, median_height, 'height')

In [ ]:
# All looks good
print('data_train_full: ', data_train_full2['height'].unique())
print('test_extrapolation: ', test_extrapolation2['height'].unique())
print('test_interpolation: ', test_interpolation2['height'].unique())

### Impute for data_train and data_val

In [ ]:
# Median height among all non-zero rows
median_height = data_train.loc[data_train['height']!=0].groupby(['ID']).height.median().median()

# Median heights by gender and ethnicity among patients with non-zero height values
height_table = pd.DataFrame({'height': data_train.loc[data_train['height']!=0].groupby(['ID']).first().groupby(['gender', 'ethnicity']).height.median()}).reset_index()
height_table

In [ ]:
data_train2 = imputer(data_train, height_table, median_height, 'height')
data_val2 = imputer(data_val, height_table, median_height, 'height')

In [ ]:
# All looks good
print('data_train: ', data_train2['height'].unique())
print('data_val: ', data_val2['height'].unique())

### Impute for CV sets

In [ ]:
train_set_list2, val_set_list2 = [], []
for i in range(len(train_set_list)):
    train_set = train_set_list[i]
    val_set = val_set_list[i]
    median_height = train_set.loc[train_set['height']!=0].groupby(['ID']).height.median().median()
    height_table = pd.DataFrame({'height': train_set.loc[train_set['height']!=0].groupby(['ID']).first().groupby(['gender', 'ethnicity']).height.median()}).reset_index()
    train_set_list2.append(imputer(train_set, height_table, median_height, 'height'))
    val_set_list2.append(imputer(val_set, height_table, median_height, 'height'))

## Weight

### Impute for data_train_full, test_extrapolation, test_interpolation

In [ ]:
# Median weight among all non-NaN rows
median_weight = data_train_full2.loc[~data_train_full2['weight'].isna()].groupby(['ID']).weight.median().median()

# Impute NaN values using the mean of each patient in the training set where available
data_train_full2['weight'] = data_train_full2['weight'].fillna(data_train_full2.groupby('ID')['weight'].transform('mean'))
test_extrapolation2['weight'] = test_extrapolation2['weight'].fillna(data_train_full2.groupby('ID')['weight'].transform('mean'))
test_interpolation2['weight'] = test_interpolation2['weight'].fillna(data_train_full2.groupby('ID')['weight'].transform('mean'))

# Fill remaining NaN values with overall median
data_train_full2['weight'].fillna(median_weight, inplace=True)
test_extrapolation2['weight'].fillna(median_weight, inplace=True)
test_interpolation2['weight'].fillna(median_weight, inplace=True)

# All looks good
print('data_train_full: ', data_train_full2['weight'].isna().sum())
print('test_extrapolation: ', test_extrapolation2['weight'].isna().sum())
print('test_interpolation: ', test_interpolation2['weight'].isna().sum())

data_train_full:  0
test_extrapolation:  0
test_extrapolation:  0


### Impute for data_train and data_val

In [ ]:
# Median weight among all non-NaN rows
median_weight = data_train2.loc[~data_train2['weight'].isna()].groupby(['ID']).weight.median().median()

# Impute NaN values using the mean of each patient in the training set where available
data_train2['weight'] = data_train2['weight'].fillna(data_train2.groupby('ID')['weight'].transform('mean'))
data_val2['weight'] = data_val2['weight'].fillna(data_train2.groupby('ID')['weight'].transform('mean'))

# Fill remaining NaN values with overall median
data_train2['weight'].fillna(median_weight, inplace=True)
data_val2['weight'].fillna(median_weight, inplace=True)

# All looks good
print('data_train: ', data_train2['weight'].isna().sum())
print('data_val: ', data_val2['weight'].isna().sum())

data_train:  0
data_val:  0


### Impute for CV sets

In [ ]:
for i in range(len(train_set_list2)):
    # Median weight among all non-NaN rows
    median_weight = train_set_list2[i].loc[~train_set_list2[i]['weight'].isna()].groupby(['ID']).weight.median().median()

    # Impute NaN values using the mean of each patient in the training set where available
    train_set_list2[i]['weight'] = train_set_list2[i]['weight'].fillna(train_set_list2[i].groupby('ID')['weight'].transform('mean'))
    val_set_list2[i]['weight'] = val_set_list2[i]['weight'].fillna(train_set_list2[i].groupby('ID')['weight'].transform('mean'))

    # Fill remaining NaN values with overall median
    train_set_list2[i]['weight'].fillna(median_weight, inplace=True)
    val_set_list2[i]['weight'].fillna(median_weight, inplace=True)

## Age

### Impute for data_train_full, test_extrapolation, test_interpolation

In [ ]:
# Median dob_year among all non-zero rows
median_dob = data_train_full2.loc[data_train_full2['dob_year']!=0].groupby(['ID']).dob_year.median().median()

# Median dob_year by gender and ethnicity among patients with non-zero dob_year values
dob_table = pd.DataFrame({'dob_year': data_train_full2.loc[data_train_full2['dob_year']!=0].groupby(['ID']).first().groupby(['gender', 'ethnicity']).dob_year.median()}).reset_index()
dob_table

NameError: ignored

In [ ]:
data_train_full3 = imputer(data_train_full2, dob_table, median_dob, 'dob_year')
test_extrapolation3 = imputer(test_extrapolation2, dob_table, median_dob, 'dob_year')
test_interpolation3 = imputer(test_interpolation2, dob_table, median_dob, 'dob_year')

NameError: ignored

In [ ]:
# Convert dob_year to age
data_train_full3.insert(3, 'age', data_train_full3['date'].dt.year - data_train_full3['dob_year'])
data_train_full3.drop(columns=['dob_year', 'date'], inplace=True)
test_extrapolation3.insert(3, 'age', test_extrapolation3['date'].dt.year - test_extrapolation3['dob_year'])
test_extrapolation3.drop(columns=['dob_year', 'date'], inplace=True)
test_interpolation3.insert(3, 'age', test_interpolation3['date'].dt.year - test_interpolation3['dob_year'])
test_interpolation3.drop(columns=['dob_year', 'date'], inplace=True)

### Impute for data_train and data_val

In [ ]:
# Median dob_year among all non-zero rows
median_dob = data_train2.loc[data_train2['dob_year']!=0].groupby(['ID']).dob_year.median().median()

# Median dob_year by gender and ethnicity among patients with non-zero dob_year values
dob_table = pd.DataFrame({'dob_year': data_train2.loc[data_train2['dob_year']!=0].groupby(['ID']).first().groupby(['gender', 'ethnicity']).dob_year.median()}).reset_index()
dob_table

In [ ]:
data_train3 = imputer(data_train2, dob_table, median_dob, 'dob_year')
data_val3 = imputer(data_val2, dob_table, median_dob, 'dob_year')

In [ ]:
# Convert dob_year to age
data_train3.insert(3, 'age', data_train3['date'].dt.year - data_train3['dob_year'])
data_train3.drop(columns=['dob_year', 'date'], inplace=True)
data_val3.insert(3, 'age', data_val3['date'].dt.year - data_val3['dob_year'])
data_val3.drop(columns=['dob_year', 'date'], inplace=True)

### Impute for CV sets

In [ ]:
train_set_list3, val_set_list3 = [], []
for i in range(len(train_set_list2)):
    train_set = train_set_list2[i]
    val_set = val_set_list2[i]
    median_dob = train_set.loc[train_set['dob_year']!=0].groupby(['ID']).dob_year.median().median()
    dob_table = pd.DataFrame({'dob_year': train_set.loc[train_set['dob_year']!=0].groupby(['ID']).first().groupby(['gender', 'ethnicity']).dob_year.median()}).reset_index()
    train_set3 = imputer(train_set, dob_table, median_dob, 'dob_year')
    val_set3 = imputer(val_set, dob_table, median_dob, 'dob_year')
    train_set3.insert(3, 'age', train_set3['date'].dt.year - train_set3['dob_year'])
    train_set3.drop(columns=['dob_year', 'date'], inplace=True)
    val_set3.insert(3, 'age', val_set3['date'].dt.year - val_set3['dob_year'])
    val_set3.drop(columns=['dob_year', 'date'], inplace=True)
    train_set_list3.append(train_set3)
    val_set_list3.append(val_set3)

## bp.sys

In [ ]:
# Fill zero values with mean of each patient
data_train_full3['bp.sys'] = data_train_full3['bp.sys'].fillna(data_train_full3.groupby('ID')['bp.sys'].transform('mean'))
test_extrapolation3['bp.sys'] = test_extrapolation3['bp.sys'].fillna(data_train_full3.groupby('ID')['bp.sys'].transform('mean'))
test_interpolation3['bp.sys'] = test_interpolation3['bp.sys'].fillna(data_train_full3.groupby('ID')['bp.sys'].transform('mean'))
data_train3['bp.sys'] = data_train3['bp.sys'].fillna(data_train3.groupby('ID')['bp.sys'].transform('mean'))
data_val3['bp.sys'] = data_val3['bp.sys'].fillna(data_train3.groupby('ID')['bp.sys'].transform('mean'))

# Fill remaining NaN with global mean
data_train_full3['bp.sys'] = data_train_full3['bp.sys'].fillna(data_train_full3['bp.sys'].mean())
test_extrapolation3['bp.sys'] = test_extrapolation3['bp.sys'].fillna(data_train_full3['bp.sys'].mean())
test_interpolation3['bp.sys'] = test_interpolation3['bp.sys'].fillna(data_train_full3['bp.sys'].mean())
data_train3['bp.sys'] = data_train3['bp.sys'].fillna(data_train3['bp.sys'].mean())
data_val3['bp.sys'] = data_val3['bp.sys'].fillna(data_train3['bp.sys'].mean())

In [ ]:
for i in range(len(train_set_list3)):
    train_set_list3[i]['bp.sys'] = train_set_list3[i]['bp.sys'].fillna(train_set_list3[i].groupby('ID')['bp.sys'].transform('mean'))
    val_set_list3[i]['bp.sys'] = val_set_list3[i]['bp.sys'].fillna(train_set_list3[i].groupby('ID')['bp.sys'].transform('mean'))
    train_set_list3[i]['bp.sys'] = train_set_list3[i]['bp.sys'].fillna(train_set_list3[i]['bp.sys'].mean())
    val_set_list3[i]['bp.sys'] = val_set_list3[i]['bp.sys'].fillna(train_set_list3[i]['bp.sys'].mean())

## bun

In [ ]:
# Fill zero values with mean of each patient
data_train_full3['bun'] = data_train_full3['bun'].fillna(data_train_full3.groupby('ID')['bun'].transform('mean'))
test_extrapolation3['bun'] = test_extrapolation3['bun'].fillna(data_train_full3.groupby('ID')['bun'].transform('mean'))
test_interpolation3['bun'] = test_interpolation3['bun'].fillna(data_train_full3.groupby('ID')['bun'].transform('mean'))
data_train3['bun'] = data_train3['bun'].fillna(data_train3.groupby('ID')['bun'].transform('mean'))
data_val3['bun'] = data_val3['bun'].fillna(data_train3.groupby('ID')['bun'].transform('mean'))

# Fill remaining NaN with global mean
data_train_full3['bun'] = data_train_full3['bun'].fillna(data_train_full3['bun'].mean())
test_extrapolation3['bun'] = test_extrapolation3['bun'].fillna(data_train_full3['bun'].mean())
test_interpolation3['bun'] = test_interpolation3['bun'].fillna(data_train_full3['bun'].mean())
data_train3['bun'] = data_train3['bun'].fillna(data_train3['bun'].mean())
data_val3['bun'] = data_val3['bun'].fillna(data_train3['bun'].mean())

In [ ]:
for i in range(len(train_set_list3)):
    train_set_list3[i]['bun'] = train_set_list3[i]['bun'].fillna(train_set_list3[i].groupby('ID')['bun'].transform('mean'))
    val_set_list3[i]['bun'] = val_set_list3[i]['bun'].fillna(train_set_list3[i].groupby('ID')['bun'].transform('mean'))
    train_set_list3[i]['bun'] = train_set_list3[i]['bun'].fillna(train_set_list3[i]['bun'].mean())
    val_set_list3[i]['bun'] = val_set_list3[i]['bun'].fillna(train_set_list3[i]['bun'].mean())

# Save

In [ ]:
# Fill NaN values with Unknown
data_train_full4 = data_train_full3.fillna('Unknown')
test_extrapolation4 = test_extrapolation3.fillna('Unknown')
test_interpolation4 = test_interpolation3.fillna('Unknown')
data_train4 = data_train3.fillna('Unknown')
data_val4 = data_val3.fillna('Unknown')

train_set_list4, val_set_list4 = [], []
for i in range(len(train_set_list3)):
    train_set_list4.append(train_set_list3[i].fillna('Unknown'))
    val_set_list4.append(val_set_list3[i].fillna('Unknown'))

In [ ]:
data_train_full4.to_csv('data/data_train_full.csv', index=False)
test_extrapolation4.to_csv('data/test_extrapolation.csv', index=False)
test_interpolation4.to_csv('data/test_interpolation.csv', index=False)
data_train4.to_csv('data/data_train.csv', index=False)
data_val4.to_csv('data/data_val.csv', index=False)

for i in range(len(train_set_list4)):
    train_set_list4[i].to_csv('data/data_cv_train_' + str(i+1) + '.csv', index=False)
    val_set_list4[i].to_csv('data/data_cv_val_' + str(i+1) + '.csv', index=False)

In [ ]:
n_samples_chosen_per_group_series = pd.Series(n_samples_chosen_per_group)
n_samples_chosen_per_group_series.to_csv('data/n_samples_chosen_per_group_series.csv', index=False)

In [ ]:
!zip -r /content/file.zip /content/data

  adding: content/data/ (stored 0%)
  adding: content/data/data_cv_val_4.csv (deflated 86%)
  adding: content/data/data_cv_train_4.csv (deflated 87%)
  adding: content/data/test_extrapolation.csv (deflated 86%)
  adding: content/data/data_cv_val_3.csv (deflated 86%)
  adding: content/data/data_cv_train_3.csv (deflated 87%)
  adding: content/data/data_cv_val_2.csv (deflated 86%)
  adding: content/data/data_cv_train_1.csv (deflated 87%)
  adding: content/data/data_train_full.csv (deflated 87%)
  adding: content/data/data_cv_train_2.csv (deflated 87%)
  adding: content/data/data_val.csv (deflated 86%)
  adding: content/data/data_cv_val_1.csv (deflated 86%)
  adding: content/data/n_samples_chosen_per_group_series.csv (deflated 82%)
  adding: content/data/data_train.csv (deflated 87%)
  adding: content/data/test_interpolation.csv (deflated 83%)
  adding: content/data/data_cv_train_5.csv (deflated 87%)
  adding: content/data/data_cv_val_5.csv (deflated 86%)
